In [2]:
import pandas as pd, numpy as np, datetime as dt, os, operator
import sqlalchemy as sql, seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [4]:
engine = sql.create_engine("sqlite:///../src/data/music.db")
with engine.connect() as con:
    attributes = pd.read_sql("SELECT * FROM attributes", con=con)

In [21]:
classifies_df = attributes.loc[:, ["danceability", "energy", "loudness", "speechiness", 'instrumentalness', 'liveness', 'valence', 'tempo', 'timesignature', 'duration',
 'explicit', "chart", 'genre_super']]

In [22]:
classifies_df = classifies_df.assign(genre_super = lambda x: x.genre_super.apply(lambda g: g if g not in ["empty", "missing"] else "other"))

In [25]:
classifies_df = classifies_df.dropna()

In [35]:
chart_test = classifies_df.iloc[:, :-1]

In [43]:
chart_test = chart_test.sample(21847, random_state=1)

In [44]:
X = chart_test.iloc[:, :-1].values
y = chart_test.iloc[:, -1].values

In [46]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X, y)

In [47]:
rf.score(X, y)

0.9989929967501259

In [48]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, y)

In [49]:
clf.score(X, y)

0.9989929967501259

In [50]:
knn = KNeighborsClassifier(2)
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [51]:
knn.score(X, y)

0.7682061610289742

In [57]:
genre_dict = {g:i for i, g in zip(range(1, 15), classifies_df.genre_super.unique())}

In [67]:
genre_test = classifies_df.assign(genre_class = lambda x: x.genre_super.apply(lambda g: genre_dict[g])).sample(21847, random_state=1)

In [69]:
y_2 = genre_test.genre_class.values
X_2 = genre_test.iloc[:, :11].values

In [72]:
rf_2 = RandomForestClassifier(n_estimators=200)
rf_2 = rf_2.fit(X_2, y_2)

In [73]:
rf_2.score(X_2, y_2)

0.9998626813750172

In [74]:
clf_2 = tree.DecisionTreeClassifier()
clf_2 = clf.fit(X_2, y_2)

In [75]:
clf_2.score(X_2, y_2)

0.9998626813750172

In [77]:
knn_2 = KNeighborsClassifier(14)
knn_2.fit(X_2, y_2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=14, p=2,
                     weights='uniform')

In [78]:
knn.score(X_2, y_2)

0.011534764498558154

In [79]:
rf_2.predict(X_2)

array([2, 5, 9, ..., 5, 2, 7], dtype=int64)

In [80]:
y_2

array([2, 5, 9, ..., 5, 2, 7], dtype=int64)